In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import KFold

In [2]:
movie = pd.read_csv('../movie.csv')

In [3]:
pd.set_option('max_column',None)

In [4]:
movie

,TITLE,DIRECTOR,OPEN_DT,OPEN_YEAR,OPEN_MONTH,OPEN_DAY,OPEN_WEEKDAY,OPEN_QUARTER,OPEN_WEEK,SHOW_TM,NATION_NM,COMPANY_NM,GENRE_NM,SP_LANG,WATCH_GRADE_NM,ACTOR1,ACTOR2,BUDGET,SERIES,KEYWORDS,AWARDS,NAVER_CMT,NAVER_CMT_NN,NAVER_PRE_EVAL,NAVER_PRE_EVAL_MUL,NAVER_EX_PT,ORI_BOOK,PLOT,AUDI_ACC,MOVIE_NM,DIRECTOR_NM
0,명량,김한민,2014-07-30,2014,7,30,2,3,31,128.0,한국,씨제이이앤엠(주),사극/액션,ja/ko,15세이상관람가,최민식,류승룡,9500000.0,0.0,사극/수군/실화바탕/실화바탕/역사물/위인/이순신/임진왜란/전쟁/해전/명량해전/명량대...,1,러일전쟁때 일본 도고제독이 한번의전투로 러시아군 5000명을 죽인 엄청난 해전이 있...,1673.0,8.64,14454.72,13639.0,1.0,1597년 임진왜란 6년 오랜 전쟁으로 인해 혼란이 극에 달한 조선. 무서운 속도로...,17613682,명량,김한민
1,극한직업,이병헌,2019-01-23,2019,1,23,2,1,4,111.0,한국,씨제이이앤엠(주),코미디,ko,15세이상관람가,류승룡,이하늬,8120000.0,0.0,잠복근무/마약단속반/위장경찰/위장취업/형사/경찰/마약범죄단/마약밀매업자/마약상/맛집...,0,스물 감독 이병헌! 영화를 매우 유쾌하게 그려낼것 같네요! 오 이병헌감독! 언제개...,189.0,8.02,1515.78,603.0,0.0,낮에는 치킨장사! 밤에는 잠복근무!지금까지 이런 수사는 없었다!불철주야 달리고 구르...,16263083,극한직업,이병헌
2,신과함께-죄와 벌,김용화,2017-12-20,2017,12,20,2,4,51,139.0,한국,롯데쇼핑㈜롯데엔터테인먼트,드라마/판타지,ko,12세이상관람가,하정우,차태현,18300000.0,1.0,웹툰원작/저승/저승사자/검사/사후세계,1,말이 필요없지.. 웹툰만큼만 해준다면 역대급 영화 극장판 애니메이션이였으면 좋겠다...,4161.0,8.04,33454.44,57194.0,1.0,승 법에 의하면 모든 인간은 사후 49일 동안 7번의 재판을 거쳐야만 한다.살인 나...,14410754,신과함께죄와벌,김용화
3,국제시장,윤제균,2014-12-17,2014,12,17,2,4,51,126.0,한국,씨제이이앤엠(주),드라마,de/en/ko/vi,12세이상관람가,황정민,김윤진,NaN,0.0,6.25/6.25/625/현대사/한국전쟁/부산/부산광역시,1,좌성향이 판치는 영화계 한줌의 빛줄기가 보인다..중립좀 지키자...산업의 역군들이시...,1229.0,8.76,10766.04,12784.0,1.0,가장 평범한 아버지의 가장 위대한 이야기’1950년대 한국전쟁 이후로부터 현재에 이...,14245998,국제시장,윤제균
4,베테랑,류승완,2015-08-05,2015,8,5,2,3,32,123.0,한국,씨제이이앤엠(주),드라마/액션,en/ko/ru,15세이상관람가,황정민,유아인,7528230.0,0.0,재벌/형사/강력반/범죄/대기업/부조리/광역수사대/재벌3세/강력계형사/실화바탕/경찰,1,걍 지릴듯... 부당거래 명콤비 ............ 류승범 도 나올려나? 유...,685.0,8.89,6089.65,9952.0,0.0,한 번 꽂힌 것은 무조건 끝을 보는 행동파 ‘서도철’(황정민)20년 경력의 승부사 ...,13395400,베테랑,류승완
5,도둑들,최동훈,2012-07-25,2012,7,25,2,3,30,135.0,한국/홍콩,(주)쇼박스,범죄/액션,cn/en/ja/ko/zh,15세이상관람가,김윤석,김혜수,NaN,0.0,다이아몬드/도둑/마카오/미술관/홍콩/범죄조직/밀매조직/조직/조직폭력/조직폭력배/보스...,1,완전 쵴코~~~!! 사람들이 재밌다는데 너무너무 궁금함 보고싶어 전지현짱 눈에...,702.0,8.69,6100.38,0.0,0.0,한치의 실수도 용납하지 않는 프로페셔널한 5인의 도둑들이 옛 보스인 마카오박으로부터...,12983330,도둑들,최동훈
6,7번방의 선물,이환경,2013-01-23,2013,1,23,2,1,4,127.0,한국,(주)넥스트엔터테인먼트월드(NEW),드라마/코미디,ko,15세이상관람가,류승룡,박신혜,NaN,0.0,공권력/교도소/누명/동료/부성애/살인누명/여자아이/출입금지/지적장애인/아빠와딸/아버...,1,7번방의 선물 충청지역 천안CGV 시사회 이벤트에 당첨되어서 18일 개봉전 먼저 보...,770.0,9.53,7338.10,0.0,0.0,흥행킹 류승룡! 세상에서 가장 사랑스러운 '딸바보'로 돌아오다! 최악의 흉악범들이 ...,12811206,7번방의선물,이환경
7,암살,최동훈,2015-07-22,2015,7,22,2,3,30,139.0,한국,(주)쇼박스,드라마/액션,cn/en/fr/ja/ko/zh,15세이상관람가,전지현,이정재,NaN,0.0,1930년대/독립군/독립운동/독립운동가/독립투사/경성/상하이/상해/의열단/저격수/항...,1,우선 배우이름으로만 500만 확정 정재리에 하대세라니;;;; 나 앓아누워요 감독님...,910.0,8.29,7543.90,20846.0,0.0,“경성에서 총 쏠 일이 있는데 이 친구들 좀 찾아와”1933년 조국이 사라진 시대 ...,12705700,암살,최동훈
8,"광해, 왕이 된 남자",추창민,2012-09-13,2012,9,13,3,3,37,131.0,한국,씨제이이앤엠(주),드라마/사극,ko,15세이상관람가,이병헌,류승룡,NaN,0.0,궁중사극/권력/권력다툼/권력싸움/대역/사극/역사/왕위/조선시대/조선왕조/폭군/취객/...,1,또 보고싶어요.꼭 상영 끝나고 DVD 나와주었으면 좋겠어요. 보고싶어요 보고싶어...,278.0,9.79,2721.62,0.0,1.0,역사에 기록되지 않은 또 한 명의 광해 왕위를 둘러싼 권력 다툼과 당쟁으로 혼란이 ...,12319542,광해왕이된남자,추창민
9,신과함께-인과 연,김용화,2018-08-01,2018,8,1,2,3,31,141.0,한국,롯데컬처웍스(주)롯데엔터테인먼트,드라마/판타지,ko,12세이상관람가,하정우,주지훈,18300000.0,1.0,웹툰원작/신과인간/저승사자/과거여행/49제/환생/사후세계/지옥/성주신/철거촌/조손가정,1,전편만큼 기대되는 작품 2번째 천만영화의 등장 도전과 발전에 한표준다 김동욱이...,484.0,6.71,3247.64,5023.0,1.0,천 년 동안 48명의 망자를 환생시킨 저승 삼차사 한 명만 더 환생시키면 그들도 새...,12253247,신과함께인과연,김용화


In [5]:
movie['PLOT'][0]

'1597년 임진왜란 6년 오랜 전쟁으로 인해 혼란이 극에 달한 조선. 무서운 속도로 한양으로 북상하는 왜군에 의해 국가존망의 위기에 처하자 누명을 쓰고 파면 당했던 이순신 장군(최민식)이 삼도수군통제사로 재임명된다. 하지만 그에게 남은 건 전의를 상실한 병사와 두려움에 가득 찬 백성 그리고 12척의 배 뿐. 마지막 희망이었던 거북선마저 불타고 잔혹한 성격과 뛰어난 지략을 지닌 용병 구루지마(류승룡)가 왜군 수장으로 나서자 조선은 더욱 술렁인다. 330척에 달하는 왜군의 배가 속속 집결하고 압도적인 수의 열세에 모두가 패배를 직감하는 순간 이순신 장군은 단 12척의 배를 이끌고 명량 바다를 향해 나서는데…! 12척의 조선 vs 330척의 왜군 역사를 바꾼 위대한 전쟁이 시작된다!'

In [6]:
vectorizer = TfidfVectorizer(
                sublinear_tf=True,
                analyzer='word',
                token_pattern=r'\w{1,}',
    )

In [7]:
overview_text = vectorizer.fit_transform(movie['PLOT'].fillna(''))

In [8]:
overview_text

<4213x82834 sparse matrix of type '<class 'numpy.float64'>'
	with 288629 stored elements in Compressed Sparse Row format>

In [9]:
word2id = defaultdict(lambda : 0)

In [10]:
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

In [18]:
for i, sent in enumerate(list(movie['PLOT'].fillna(''))):
    if i >=5:
        break
    print('====== document[%d] ======' % i)
    print( [ (token, overview_text[i, word2id[token]]) for token in sent.split() ])

====== document[0] ======
[('1597년', array(0.12307393)), ('임진왜란', array(0.11321514)), ('6년', array(0.09105328)), ('오랜', array(0.0690898)), ('전쟁으로', array(0.08833586)), ('인해', array(0.05509934)), ('혼란이', array(0.11730691)), ('극에', array(0.09645085)), ('달한', array(0.11004132)), ('조선.', array(0.)), ('무서운', array(0.07847707)), ('속도로', array(0.10168109)), ('한양으로', array(0.11321514)), ('북상하는', array(0.12307393)), ('왜군에', array(0.12307393)), ('의해', array(0.06507816)), ('국가존망의', array(0.12307393)), ('위기에', array(0.05815414)), ('처하자', array(0.10168109)), ('누명을', array(0.0815738)), ('쓰고', array(0.08409032)), ('파면', array(0.11730691)), ('당했던', array(0.11730691)), ('이순신', array(0.19861786)), ('장군(최민식)이', array(0.)), ('삼도수군통제사로', array(0.12307393)), ('재임명된다.', array(0.)), ('하지만', array(0.03643581)), ('그에게', array(0.06234503)), ('남은', array(0.06569399)), ('건', array(0.05577239)), ('전의를', array(0.11730691)), ('상실한', array(0.10525559)), ('병사와', array(0.12307393)), ('두려움에', array(0.0953968)), ('가득', ar

In [19]:
movie

,TITLE,DIRECTOR,OPEN_DT,OPEN_YEAR,OPEN_MONTH,OPEN_DAY,OPEN_WEEKDAY,OPEN_QUARTER,OPEN_WEEK,SHOW_TM,NATION_NM,COMPANY_NM,GENRE_NM,SP_LANG,WATCH_GRADE_NM,ACTOR1,ACTOR2,BUDGET,SERIES,KEYWORDS,AWARDS,NAVER_CMT,NAVER_CMT_NN,NAVER_PRE_EVAL,NAVER_PRE_EVAL_MUL,NAVER_EX_PT,ORI_BOOK,PLOT,AUDI_ACC,MOVIE_NM,DIRECTOR_NM
0,명량,김한민,2014-07-30,2014,7,30,2,3,31,128.0,한국,씨제이이앤엠(주),사극/액션,ja/ko,15세이상관람가,최민식,류승룡,9500000.0,0.0,사극/수군/실화바탕/실화바탕/역사물/위인/이순신/임진왜란/전쟁/해전/명량해전/명량대...,1,러일전쟁때 일본 도고제독이 한번의전투로 러시아군 5000명을 죽인 엄청난 해전이 있...,1673.0,8.64,14454.72,13639.0,1.0,1597년 임진왜란 6년 오랜 전쟁으로 인해 혼란이 극에 달한 조선. 무서운 속도로...,17613682,명량,김한민
1,극한직업,이병헌,2019-01-23,2019,1,23,2,1,4,111.0,한국,씨제이이앤엠(주),코미디,ko,15세이상관람가,류승룡,이하늬,8120000.0,0.0,잠복근무/마약단속반/위장경찰/위장취업/형사/경찰/마약범죄단/마약밀매업자/마약상/맛집...,0,스물 감독 이병헌! 영화를 매우 유쾌하게 그려낼것 같네요! 오 이병헌감독! 언제개...,189.0,8.02,1515.78,603.0,0.0,낮에는 치킨장사! 밤에는 잠복근무!지금까지 이런 수사는 없었다!불철주야 달리고 구르...,16263083,극한직업,이병헌
2,신과함께-죄와 벌,김용화,2017-12-20,2017,12,20,2,4,51,139.0,한국,롯데쇼핑㈜롯데엔터테인먼트,드라마/판타지,ko,12세이상관람가,하정우,차태현,18300000.0,1.0,웹툰원작/저승/저승사자/검사/사후세계,1,말이 필요없지.. 웹툰만큼만 해준다면 역대급 영화 극장판 애니메이션이였으면 좋겠다...,4161.0,8.04,33454.44,57194.0,1.0,승 법에 의하면 모든 인간은 사후 49일 동안 7번의 재판을 거쳐야만 한다.살인 나...,14410754,신과함께죄와벌,김용화
3,국제시장,윤제균,2014-12-17,2014,12,17,2,4,51,126.0,한국,씨제이이앤엠(주),드라마,de/en/ko/vi,12세이상관람가,황정민,김윤진,NaN,0.0,6.25/6.25/625/현대사/한국전쟁/부산/부산광역시,1,좌성향이 판치는 영화계 한줌의 빛줄기가 보인다..중립좀 지키자...산업의 역군들이시...,1229.0,8.76,10766.04,12784.0,1.0,가장 평범한 아버지의 가장 위대한 이야기’1950년대 한국전쟁 이후로부터 현재에 이...,14245998,국제시장,윤제균
4,베테랑,류승완,2015-08-05,2015,8,5,2,3,32,123.0,한국,씨제이이앤엠(주),드라마/액션,en/ko/ru,15세이상관람가,황정민,유아인,7528230.0,0.0,재벌/형사/강력반/범죄/대기업/부조리/광역수사대/재벌3세/강력계형사/실화바탕/경찰,1,걍 지릴듯... 부당거래 명콤비 ............ 류승범 도 나올려나? 유...,685.0,8.89,6089.65,9952.0,0.0,한 번 꽂힌 것은 무조건 끝을 보는 행동파 ‘서도철’(황정민)20년 경력의 승부사 ...,13395400,베테랑,류승완
5,도둑들,최동훈,2012-07-25,2012,7,25,2,3,30,135.0,한국/홍콩,(주)쇼박스,범죄/액션,cn/en/ja/ko/zh,15세이상관람가,김윤석,김혜수,NaN,0.0,다이아몬드/도둑/마카오/미술관/홍콩/범죄조직/밀매조직/조직/조직폭력/조직폭력배/보스...,1,완전 쵴코~~~!! 사람들이 재밌다는데 너무너무 궁금함 보고싶어 전지현짱 눈에...,702.0,8.69,6100.38,0.0,0.0,한치의 실수도 용납하지 않는 프로페셔널한 5인의 도둑들이 옛 보스인 마카오박으로부터...,12983330,도둑들,최동훈
6,7번방의 선물,이환경,2013-01-23,2013,1,23,2,1,4,127.0,한국,(주)넥스트엔터테인먼트월드(NEW),드라마/코미디,ko,15세이상관람가,류승룡,박신혜,NaN,0.0,공권력/교도소/누명/동료/부성애/살인누명/여자아이/출입금지/지적장애인/아빠와딸/아버...,1,7번방의 선물 충청지역 천안CGV 시사회 이벤트에 당첨되어서 18일 개봉전 먼저 보...,770.0,9.53,7338.10,0.0,0.0,흥행킹 류승룡! 세상에서 가장 사랑스러운 '딸바보'로 돌아오다! 최악의 흉악범들이 ...,12811206,7번방의선물,이환경
7,암살,최동훈,2015-07-22,2015,7,22,2,3,30,139.0,한국,(주)쇼박스,드라마/액션,cn/en/fr/ja/ko/zh,15세이상관람가,전지현,이정재,NaN,0.0,1930년대/독립군/독립운동/독립운동가/독립투사/경성/상하이/상해/의열단/저격수/항...,1,우선 배우이름으로만 500만 확정 정재리에 하대세라니;;;; 나 앓아누워요 감독님...,910.0,8.29,7543.90,20846.0,0.0,“경성에서 총 쏠 일이 있는데 이 친구들 좀 찾아와”1933년 조국이 사라진 시대 ...,12705700,암살,최동훈
8,"광해, 왕이 된 남자",추창민,2012-09-13,2012,9,13,3,3,37,131.0,한국,씨제이이앤엠(주),드라마/사극,ko,15세이상관람가,이병헌,류승룡,NaN,0.0,궁중사극/권력/권력다툼/권력싸움/대역/사극/역사/왕위/조선시대/조선왕조/폭군/취객/...,1,또 보고싶어요.꼭 상영 끝나고 DVD 나와주었으면 좋겠어요. 보고싶어요 보고싶어...,278.0,9.79,2721.62,0.0,1.0,역사에 기록되지 않은 또 한 명의 광해 왕위를 둘러싼 권력 다툼과 당쟁으로 혼란이 ...,12319542,광해왕이된남자,추창민
9,신과함께-인과 연,김용화,2018-08-01,2018,8,1,2,3,31,141.0,한국,롯데컬처웍스(주)롯데엔터테인먼트,드라마/판타지,ko,12세이상관람가,하정우,주지훈,18300000.0,1.0,웹툰원작/신과인간/저승사자/과거여행/49제/환생/사후세계/지옥/성주신/철거촌/조손가정,1,전편만큼 기대되는 작품 2번째 천만영화의 등장 도전과 발전에 한표준다 김동욱이...,484.0,6.71,3247.64,5023.0,1.0,천 년 동안 48명의 망자를 환생시킨 저승 삼차사 한 명만 더 환생시키면 그들도 새...,12253247,신과함께인과연,김용화


In [22]:
movie_text = movie.loc[:,['TITLE','DIRECTOR','OPEN_DT','KEYWORDS','NAVER_CMT','PLOT','AUDI_ACC']]

In [23]:
movie.columns

Index(['TITLE', 'DIRECTOR', 'OPEN_DT', 'OPEN_YEAR', 'OPEN_MONTH', 'OPEN_DAY',
       'OPEN_WEEKDAY', 'OPEN_QUARTER', 'OPEN_WEEK', 'SHOW_TM', 'NATION_NM',
       'COMPANY_NM', 'GENRE_NM', 'SP_LANG', 'WATCH_GRADE_NM', 'ACTOR1',
       'ACTOR2', 'BUDGET', 'SERIES', 'KEYWORDS', 'AWARDS', 'NAVER_CMT',
       'NAVER_CMT_NN', 'NAVER_PRE_EVAL', 'NAVER_PRE_EVAL_MUL', 'NAVER_EX_PT',
       'ORI_BOOK', 'PLOT', 'AUDI_ACC', 'MOVIE_NM', 'DIRECTOR_NM'],
      dtype='object')

In [25]:
movie_regression = movie.drop(['KEYWORDS','NAVER_CMT','PLOT','MOVIE_NM', 'DIRECTOR_NM'],axis=1)

In [27]:
movie_regression.to_csv('movie_regression.csv',index=False, encoding='cp949')

In [28]:
movie_text.to_csv()

,TITLE,DIRECTOR,OPEN_DT,KEYWORDS,NAVER_CMT,PLOT,AUDI_ACC
0,명량,김한민,2014-07-30,사극/수군/실화바탕/실화바탕/역사물/위인/이순신/임진왜란/전쟁/해전/명량해전/명량대...,러일전쟁때 일본 도고제독이 한번의전투로 러시아군 5000명을 죽인 엄청난 해전이 있...,1597년 임진왜란 6년 오랜 전쟁으로 인해 혼란이 극에 달한 조선. 무서운 속도로...,17613682
1,극한직업,이병헌,2019-01-23,잠복근무/마약단속반/위장경찰/위장취업/형사/경찰/마약범죄단/마약밀매업자/마약상/맛집...,스물 감독 이병헌! 영화를 매우 유쾌하게 그려낼것 같네요! 오 이병헌감독! 언제개...,낮에는 치킨장사! 밤에는 잠복근무!지금까지 이런 수사는 없었다!불철주야 달리고 구르...,16263083
2,신과함께-죄와 벌,김용화,2017-12-20,웹툰원작/저승/저승사자/검사/사후세계,말이 필요없지.. 웹툰만큼만 해준다면 역대급 영화 극장판 애니메이션이였으면 좋겠다...,승 법에 의하면 모든 인간은 사후 49일 동안 7번의 재판을 거쳐야만 한다.살인 나...,14410754
3,국제시장,윤제균,2014-12-17,6.25/6.25/625/현대사/한국전쟁/부산/부산광역시,좌성향이 판치는 영화계 한줌의 빛줄기가 보인다..중립좀 지키자...산업의 역군들이시...,가장 평범한 아버지의 가장 위대한 이야기’1950년대 한국전쟁 이후로부터 현재에 이...,14245998
4,베테랑,류승완,2015-08-05,재벌/형사/강력반/범죄/대기업/부조리/광역수사대/재벌3세/강력계형사/실화바탕/경찰,걍 지릴듯... 부당거래 명콤비 ............ 류승범 도 나올려나? 유...,한 번 꽂힌 것은 무조건 끝을 보는 행동파 ‘서도철’(황정민)20년 경력의 승부사 ...,13395400
5,도둑들,최동훈,2012-07-25,다이아몬드/도둑/마카오/미술관/홍콩/범죄조직/밀매조직/조직/조직폭력/조직폭력배/보스...,완전 쵴코~~~!! 사람들이 재밌다는데 너무너무 궁금함 보고싶어 전지현짱 눈에...,한치의 실수도 용납하지 않는 프로페셔널한 5인의 도둑들이 옛 보스인 마카오박으로부터...,12983330
6,7번방의 선물,이환경,2013-01-23,공권력/교도소/누명/동료/부성애/살인누명/여자아이/출입금지/지적장애인/아빠와딸/아버...,7번방의 선물 충청지역 천안CGV 시사회 이벤트에 당첨되어서 18일 개봉전 먼저 보...,흥행킹 류승룡! 세상에서 가장 사랑스러운 '딸바보'로 돌아오다! 최악의 흉악범들이 ...,12811206
7,암살,최동훈,2015-07-22,1930년대/독립군/독립운동/독립운동가/독립투사/경성/상하이/상해/의열단/저격수/항...,우선 배우이름으로만 500만 확정 정재리에 하대세라니;;;; 나 앓아누워요 감독님...,“경성에서 총 쏠 일이 있는데 이 친구들 좀 찾아와”1933년 조국이 사라진 시대 ...,12705700
8,"광해, 왕이 된 남자",추창민,2012-09-13,궁중사극/권력/권력다툼/권력싸움/대역/사극/역사/왕위/조선시대/조선왕조/폭군/취객/...,또 보고싶어요.꼭 상영 끝나고 DVD 나와주었으면 좋겠어요. 보고싶어요 보고싶어...,역사에 기록되지 않은 또 한 명의 광해 왕위를 둘러싼 권력 다툼과 당쟁으로 혼란이 ...,12319542
9,신과함께-인과 연,김용화,2018-08-01,웹툰원작/신과인간/저승사자/과거여행/49제/환생/사후세계/지옥/성주신/철거촌/조손가정,전편만큼 기대되는 작품 2번째 천만영화의 등장 도전과 발전에 한표준다 김동욱이...,천 년 동안 48명의 망자를 환생시킨 저승 삼차사 한 명만 더 환생시키면 그들도 새...,12253247
